In [1]:
!pip install -q accelerate datasets peft bitsandbytes tensorboard
!pip install -q flash-attn --no-build-isolation
!pip install evaluate
!pip install rouge_score


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 16.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=1eac8a4d64ac6814cf613122adce9a064a3e972af1b36f8fa2738e7a6a7b34bb
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics3ForConditionalGeneration

In [4]:
USE_LORA = False
USE_QLORA = True
SMOL = True

In [5]:
model_id = "HuggingFaceTB/SmolVLM-Base" if SMOL else "HuggingFaceM4/Idefics3-8B-Llama3"

In [6]:
processor = AutoProcessor.from_pretrained(
    model_id
)

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/424 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/486 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.05k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

In [7]:
if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    lora_config.inference_mode = False
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_id,
        quantization_config=bnb_config if USE_QLORA else None,
        _attn_implementation="flash_attention_2",
        device_map="auto"
    )
    model.add_adapter(lora_config)
    model.enable_adapters()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    print(model.get_nb_trainable_parameters())
else:
    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        _attn_implementation="flash_attention_2",
    ).to(DEVICE)

    # if you'd like to only fine-tune LLM
    for param in model.model.vision_model.parameters():
        param.requires_grad = False

config.json:   0%|          | 0.00/7.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

(10536960, 2256809840)


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [8]:
from datasets import load_dataset
ds = load_dataset('TrikyViktor/Anime-Image-Captioning', trust_remote_code=True)
ds = ds['train'].train_test_split(test_size=0.1, seed=42)

README.md:   0%|          | 0.00/312 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.43M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/56 [00:00<?, ? examples/s]

In [9]:
ds['train'][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=722x425>,
 'caption': 'Itadori and Todou standing next to each other, with todou standing shirtless while itadori is with his regular purple clothe.'}

In [10]:
ds['test'][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=728x410>,
 'caption': "Sasuke and Naruto confronts each other gripping each other's clothes. Sasuke has a black hair while Naruto has a blond hair."}

In [11]:
image_token_id = processor.tokenizer.additional_special_tokens_ids[
            processor.tokenizer.additional_special_tokens.index("<image>")]

In [12]:
def collate_fn(examples):
  texts = []
  images = []
  for example in examples:
      image = example["image"]
      if image.mode != 'RGB':
        image = image.convert('RGB')
      answer = example["caption"]
      question = "What is happening in the image?"
      messages = [
          {
              "role": "user",
              "content": [
                  {"type": "text", "text": "Answer briefly."},
                  {"type": "image"},
                  {"type": "text", "text": question}
              ]
          },
          {
              "role": "assistant",
              "content": [
                  {"type": "text", "text": answer}
              ]
          }
      ]
      text = processor.apply_chat_template(messages, add_generation_prompt=False)
      texts.append(text.strip())
      images.append([image])

  batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
  labels = batch["input_ids"].clone()
  labels[labels == processor.tokenizer.pad_token_id] = -100
  labels[labels == image_token_id] = -100
  batch["labels"] = labels

  return batch

In [13]:
from transformers import TrainingArguments, Trainer

model_name = model_id.split("/")[-1]

training_args = TrainingArguments(
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=2,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=1,
    optim="paged_adamw_8bit", # for 8-bit, keep this, else adamw_hf
    bf16=True, # underlying precision for 8bit
    output_dir=f"./{model_name}-vqav2",
    hub_model_id=f"{model_name}-vqav2",
    report_to="tensorboard",
    remove_unused_columns=False,
    gradient_checkpointing=True
)

In [14]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    decoded_preds = processor.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = processor.batch_decode(labels, skip_special_tokens=True)

    # Strip white spaces
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [label.strip() for label in decoded_labels]

    # Wrap references for BLEU
    bleu_result = bleu.compute(predictions=decoded_preds, references=[[ref] for ref in decoded_labels])
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    return {
        "bleu": bleu_result.get("bleu", 0.0),
        "rougeL": rouge_result.get("rougeL", 0.0)
    }

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=ds['train'],
    eval_dataset=ds['test'],
    compute_metrics=compute_metrics,
)

No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [17]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
2,1.417000
4,0.874800
6,1.373600
8,0.845600
10,1.313100
12,0.794400
14,1.212300
16,0.702400
18,1.074300
20,0.645500


TrainOutput(global_step=30, training_loss=0.9302193959554036, metrics={'train_runtime': 367.4437, 'train_samples_per_second': 1.361, 'train_steps_per_second': 0.082, 'total_flos': 7112543680814208.0, 'train_loss': 0.9302193959554036, 'epoch': 7.615384615384615})

In [18]:
# Save model and processor
model.save_pretrained("my_finetuned_smolvlm")
processor.save_pretrained("my_finetuned_smolvlm")

# push to the Hugging Face Hub
trainer.push_to_hub()

print("Fine-tuning complete!")

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/42.2M [00:00<?, ?B/s]

events.out.tfevents.1744242227.1283d74f07d6.372.0:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

Fine-tuning complete!


In [19]:
metrics = trainer.evaluate()
print(metrics)

{'eval_runtime': 3.7757, 'eval_samples_per_second': 1.589, 'eval_steps_per_second': 0.265, 'epoch': 7.615384615384615}


In [20]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="TrikyViktor/SmolVLM-Base-vqav2",
    folder_path="my_finetuned_smolvlm",
    repo_type="model"
)

CommitInfo(commit_url='https://huggingface.co/TrikyViktor/SmolVLM-Base-vqav2/commit/57b1f3ffb677e302d332983b5dd1ebbeae6e1d31', commit_message='Upload folder using huggingface_hub', commit_description='', oid='57b1f3ffb677e302d332983b5dd1ebbeae6e1d31', pr_url=None, repo_url=RepoUrl('https://huggingface.co/TrikyViktor/SmolVLM-Base-vqav2', endpoint='https://huggingface.co', repo_type='model', repo_id='TrikyViktor/SmolVLM-Base-vqav2'), pr_revision=None, pr_num=None)